In [1]:
from scipy import stats
import math
import glob
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from skimage import io
import glob
from skimage.filters import threshold_local
from skimage.filters import threshold_li
from skimage.color import rgb2gray
from math import sqrt
from skimage.morphology import disk, ball
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.filters.rank import enhance_contrast
from skimage.exposure import adjust_gamma
import pickle as pkl
import sys
from datetime import date
import os, psutil
import re
process = psutil.Process(os.getpid())
# Grab utilities from the imaging directories dir
utilsDir = re.sub(r'Registration', 'Imaging Utilities', str(sys.path[0]))
# Homebrew utilities for importing ims files & basic image manipulation
sys.path.append(utilsDir)
from functools import reduce
from scipy.spatial.distance import cdist, pdist
from skimage.exposure import rescale_intensity
from skimage.segmentation import find_boundaries

# Use this function for local thresholding of images
def lthresh(imgstk):
    th = threshold_local(imgstk, 65, offset=0)
    imgstk = imgstk>= th
    return imgstk
# Use this function to threshold images
def thresh(imgstk):
    th = threshold_li(imgstk)
    imgstk = imgstk>= th
    return imgstk
def spotcall(img, min_sig, max_sig, nsig, th):
    blobs_log = blob_log(img, min_sigma=min_sig, max_sigma=max_sig, num_sigma=nsig, threshold=th)
    # Compute radii in the 3rd column.
    blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
    color = 'lime'
    title = 'Laplacian of Gaussian'
    return blobs_log

In [2]:
import pandas as pd
codebook = '/mnt/disks/external/jg4159/BEpilot/DDR364_codebook.csv'
cb = pd.read_csv(codebook, sep=',', header=0, index_col='Gene')
print(cb)
cb_list = np.asarray(cb.values.tolist(),dtype=bool)
genes = cb.index.tolist()
print(np.asarray(cb_list[0]).shape)

                          R1  R2  R3  R4  R5  R6  R7  R8  R9  R10  ...  R15  \
Gene                                                               ...        
AAVS1-controls-AAVS1.100   0   1   0   0   0   0   0   0   0    0  ...    0   
AAVS1-controls-AAVS1.103   0   0   0   0   0   0   0   0   0    0  ...    0   
AAVS1-controls-AAVS1.105   0   0   0   1   0   0   0   1   0    0  ...    0   
AAVS1-controls-AAVS1.106   0   0   0   0   0   0   1   0   0    0  ...    0   
AAVS1-controls-AAVS1.108   1   0   0   0   0   0   0   0   0    0  ...    0   
...                       ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...   
Core-RAD51D.51             1   0   0   0   0   1   0   0   1    0  ...    0   
Core-RECQL.109             1   0   0   0   0   0   0   0   1    0  ...    1   
Core-RECQL.71              1   0   0   0   0   0   0   0   1    0  ...    0   
Core-WRN.125               1   0   1   0   0   0   0   0   1    0  ...    0   
Core-XRCC3.198             0   0   0   0   0   0   0

In [ ]:
runlist= ['P1_1','P1_2','P1_3','P1_4','P1_5','P1_6'] #,'P1_7','P1_8','P1_9','P2_1','P2_2','P2_3','P2_4','P2_5', 'P2_6','P2_7','P2_8','P2_9'
opdir = '/mnt/disks/external/jg4159/20231006_DDR364/' # Path to output 
for runName in runlist:
    print(runName)
    ip = opdir + "RegisteredImages_CM_" + runName + ".pkl" # the registered image stack containing all CM rounds
    filehandler = open(ip, 'rb')
    img_stack = pkl.load(filehandler)
    filehandler.close()
    mk = opdir + "Unfiltered_segmentation_DDR364_"+runName+"*.pickle" # the segmentation files
    lof = glob.glob(mk)
    filehandler = open(lof[0], 'rb')
    masks_mem,masks_nuc,ref_mem = pkl.load(filehandler)
    print(len(ref_mem))
    filehandler.close()

    # rescale the registered images to unit16
    for img in img_stack:
        for i in range(img.shape[0]):
            img[i] *= 65535
            img[i] = img[i].astype(np.uint16)
            m = np.median(img[i][img[i] > 0])
            
    # rescale the images by setting lower and upper thresholds
    num_chn = 3
    num_cyc = 8

    gamstack=[]
    for i in range(num_cyc):
            for j in range(num_chn):
                if (j<2): #640 or 561 channel
                    gamstack.append(rescale_intensity(img_stack[i][j], (108,115), (0, 255)))
                else: #488 channel
                    gamstack.append(rescale_intensity(img_stack[i][j],(108,112), (0, 255)))

    # check if memMaskgd exists
    try:
        filehandler = open(opdir+'memMaskgd0_reverse_' + runName + '.pkl', 'rb')
        memMaskgd0 = pkl.load(filehandler)
        filehandler.close()
    except:
        memMaskgd_exists = False
    else:
        print("Exists!")
        memMaskgd_exists = True
    # make config if it does not exist already (e.g. passed in by papermill) for manual running
    if not(memMaskgd_exists):
        start_time = time.time()
        memMaskgd0 = {}
        i=0
        for key in ref_mem.keys():
            if i%1000==0:
                print(i)
            i=i+1
            spgd={}
            x0 = ref_mem[key]["Membrane Pixels 2D"][0]
            y0 = ref_mem[key]["Membrane Pixels 2D"][1]
            cellset = set([tuple([x,y]) for x,y in zip(x0,y0)]) #save all pixels for this cell\n",
            spgd['cellset']=cellset
            memMaskgd0[key] = spgd
        print(f'--- generate memeMaskgd0: {(time.time()-start_time)} seconds')
        filehandler = open(opdir+'memMaskgd0_reverse_' + runName + '.pkl', 'wb')
        pkl.dump(memMaskgd0, filehandler)
        filehandler.close()
        
    # spot calling 
    spot_df = pd.DataFrame(0,columns = range(len(gamstack)), 
                       index = memMaskgd0.keys())
    all_spots = np.zeros(shape=(0,3))
    total_spots_cycle = []
    retained_spots_cycle = []
    spotlist = []
    bkg_spot_th = 3 # remove scattered spots to reduce the spot matrix density
    cyc=0
    for img in gamstack:
        spots = spotcall(img[:,:],min_sig=2,max_sig=10,nsig=1,th=0.5)# Decrease threshold if spots are not identified and min sigma if spots are smaller\n",
        spotset = set([tuple(x) for x in spots[:,:2].astype(int)])
        retained_spots= []
        num_retained = 0
        for key, values in memMaskgd0.items():
            cellset=memMaskgd0[key]["cellset"]  # all 2d pixel positions for this mem id    
            spcell = np.array([[x[0],x[1],1] for x in spotset & cellset]) # find pixels (i.e. spots) that are in both spotset & cellset
            spot_df.loc[key, cyc] = len(spcell)
            if len(spcell) >= bkg_spot_th:
                num_retained = num_retained + len(spcell)
                spcell = spcell.tolist()
                retained_spots.extend(spcell)
        spotlist.append(np.array(retained_spots))
        total_spots_cycle.append(len(spots))
        retained_spots_cycle.append(len(retained_spots))
        all_spots = np.concatenate((all_spots, retained_spots), axis=0)
        cyc=cyc+1
    all_spots = np.unique(all_spots, axis=0)
    print("Total # of unique spots:", all_spots.shape[0])
    total_spots = all_spots.shape[0]
    # save the spot coordinates
    filehandler = open(opdir+'Spotcalling_spotlist_108115&108112_' + str(total_spots) + "_" + runName + '.pkl', 'wb')
    pkl.dump(spotlist, filehandler)
    filehandler.close()
    filehandler = open(opdir+'Spotcalling_allspots_108115&108112_' + str(total_spots) + "_"  + runName + '.pkl', 'wb')
    pkl.dump(all_spots, filehandler)
    filehandler.close()
    spot_df.to_csv(opdir+runName+'_'+str(total_spots)+'_108115&108112_spotbycycle.csv')
    
    # Amplicon decoding
    xmax = gamstack[0].shape[0]
    ymax = gamstack[0].shape[1]
    del gamstack
    spotset=[]
    for j in spotlist:
        spotset.append(set([tuple(x) for x in j[:,:2]]))
    num_chn=3
    num_cyc=8
    num_pxl_list=[2.01]#test different radius if necessary
    guide_spots_list=[]
    for num_pxl in num_pxl_list:    
        ## Guide detection, generating the bitcode by searching the spot pixel in all cycles within a given radius
        cycles = np.zeros((all_spots.shape[0],num_chn*num_cyc),dtype=bool)
        for i in range(all_spots.shape[0]): #all_spots.shape[0]
            k = 0
            # given a spot i
            spot_i = np.reshape(all_spots[i,:2],[1,2])[0]
            grid = [] 
            for dx in range(math.ceil(num_pxl)):  # give the radius is 2, we can pre-set a coordinate set of 2x2 grid and find spots that fall within the grid, calculate distance if any
                for dy in range(math.ceil(num_pxl)):
                    coord1 = tuple([np.max([0,spot_i[0]-dx]).astype(int), np.max([0,spot_i[1]-dy]).astype(int)])
                    #print(coord1)
                    grid.append(coord1)
                    coord2 = tuple([np.min([xmax, spot_i[0]+dx]).astype(int), np.min([ymax, spot_i[1]+dy]).astype(int)])
                    #print(coord2)
                    grid.append(coord2)
                    coord3 = tuple([np.max([0, spot_i[0]-dx]).astype(int), np.min([ymax, spot_i[1]+dy]).astype(int)])
                    grid.append(coord3)
                    coord4 = tuple([np.min([xmax, spot_i[0]+dx]).astype(int), np.max([0, spot_i[1]-dy]).astype(int)])
                    grid.append(coord4)
            grid = set(grid)      
            for j in spotset:  
                spcell = np.array([[x[0],x[1]] for x in grid & j])
                if len(spcell>0): # if there is a spot within this grid
                    a = cdist(np.reshape(all_spots[i,:2],[1,2]), spcell, metric='euclidean') # calculate the cdist of the current spot to all the other spots found in the grid
                    if np.min(a) < num_pxl: # if the cdist is less than the num_pxl pixels 
                        cycles[i,k]=1
                k = k+1

    # de-duplicaton
        spotdict = {}
        guide_spots=0
        for j in range(len(cb_list)):
            k = 0
            abc = []
            dedup = []
            for i in range(0, cycles.shape[0]):
                if np.array_equal(cycles[i,:], np.asarray(cb_list[j])):
                    k=k+1
                    abc.append(all_spots[i])
            spotcycle = np.asarray(abc)
            for sp in range(spotcycle.shape[0]):
                a = cdist(np.reshape(spotcycle[sp,:2],[1,2]), spotcycle[:,:2], metric='euclidean')#distance
                if len(np.where(a<num_pxl)[1]) > 1:
                    dedup.append(spotcycle[np.where(a<num_pxl)[1][0],:])
                else:
                    dedup.append(spotcycle[sp,:])
            spotdict[genes[j]] = np.unique(np.asarray(dedup),axis=0)
            guide_spots = guide_spots + len(dedup)
        print("Total guide spots: ", guide_spots)
        print("Guide spots/Total spots:", guide_spots, '/', total_spots, '=', guide_spots/total_spots)
        guide_spots_list.append(guide_spots)
        # Save the guide spot ds' coordinates
        filehandler = open(opdir+'Spots-with-dist-108115&108112_numpxl='+str(num_pxl)+"_"+str(guide_spots)+"_" + runName + '.pkl', 'wb')
        pkl.dump(spotdict, filehandler)
        filehandler.close()

        # Create a dictionary where every key is a foci coordinate, and the value is the guide identity
        spotMatch = {}
        names = list(spotdict.keys())
        for i, guide in enumerate(spotdict):
            for spot in spotdict[guide]:
                if spotMatch.get((spot[0], spot[1])):
                    spotMatch[(spot[0], spot[1])].append(names[i])
                else:
                    spotMatch[(spot[0], spot[1])] = [names[i]]

        # create an empty dataframe with cell_id and guide names
        mem_df = pd.DataFrame(0,columns = spotdict.keys(), 
                           index = memMaskgd0.keys())        

        # For each nuclei, see which spots in the membrane are indexed in the key 
        for i, cellID in enumerate(memMaskgd0):
            for name in names:
                memMaskgd0[cellID][name] = 0

            # For each Mmebrane Coordinate set, look for a spot
            memCoord = memMaskgd0[cellID]['cellset']
            for coord in memCoord:
                amplicons = spotMatch.get(coord)
                if amplicons:
                    for amplicon in amplicons:
                        memMaskgd0[cellID][amplicon] += 1
                        mem_df.loc[cellID,amplicon] += 1

        mem_max = mem_df.max(axis='columns') # guide with the max num of spots
        mem_sec = mem_df.apply(lambda x: x.nlargest(2).iloc[1], axis=1)
        max_idx = mem_df.idxmax(axis=1)
        guide_num = 0
        for i, cellID in enumerate(ref_mem):
            guide_spot_ratio = mem_max[cellID]/(mem_max[cellID] + mem_sec[cellID]) # guide purity
            if mem_max[cellID] >= 3 and guide_spot_ratio >= 0.67:
                ref_mem[cellID]['Guide ID'] = max_idx[cellID]
                guide_num = guide_num + 1
            else:
                ref_mem[cellID]['Guide ID'] = "None"
        print(guide_num)
        print(guide_num/len(ref_mem))
        mem_df.to_csv(opdir+runName+'_108115&108112_'+str(guide_spots)+"spots_"+str(guide_num)+'cells_cell2guide.csv')
        # check nucleus presence in all cycles and save as an entry in ref_mem
        th = 105
        for key in ref_mem.keys(): 
            round_sum = 0
            # Get Indices of Nuclei volume & append to dictionary
            nuc_pxls = ref_mem[key]['Nuclei Pixels']
            # Get the most common membrane mask value for that location in a given round
            for i in range(len(img_stack)):
                x=img_stack[i][3].ravel()
                x = x > th # binarized
                cur_id = stats.mode(x[nuc_pxls])[0] # 1 or 0
                round_sum = round_sum + cur_id
            # Save within a sub dictionary
            sub_dict1 = dict()
            sub_dict1['round_sum'] = round_sum
            # save the sub dictionary
            ref_mem[key].update(sub_dict1)
        # save the dict with guide and roundsum info
        filehandler = open(opdir+'ref_mem_roundsum_108115&108112_' + runName + '.pkl', 'wb')
        pkl.dump((masks_mem,masks_nuc,ref_mem), filehandler)
        filehandler.close()

P1_1
14793
Exists!


/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:125: RuntimeWarning: invalid value encountered in double_scalars
  r1 = blob1[-1] / blob2[-1]
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:126: RuntimeWarning: divide by zero encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:127: RuntimeWarning: divide by zero encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:129: RuntimeWarning: invalid value encountered in subtract
  d = np.sqrt(np.sum((pos2 - pos1)**2))
/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/skimage/feature/blob.py:126: RuntimeWarning: invalid value encountered in true_divide
  pos1 = blob1[:ndim] /

Total # of unique spots: 655716
Total guide spots:  207891
Guide spots/Total spots: 207891 / 655716 = 0.31704426916530937


/mnt/disks/external/jg4159/.conda/envs/cellpose/lib/python3.7/site-packages/ipykernel_launcher.py:205: RuntimeWarning: invalid value encountered in long_scalars


9175
0.6202257824646792
P1_2
22485
Exists!
Total # of unique spots: 1237543
Total guide spots:  249157
Guide spots/Total spots: 249157 / 1237543 = 0.20133199412060834
12233
0.5440515899488548
